In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.mllib.stat import Statistics

In [2]:
spark = (SparkSession.builder.appName("Day 9").master("local[1]").enableHiveSupport().getOrCreate())

## Data Preparation

### Bank Analysis

<b>Context</b>

Banks play a crucial role in market economies. They decide who can obtain financing and on what terms and can make or break investment decisions. For markets and society to function, individuals and companies need access to credit. 
 
Banks use credit scoring algorithms to determine whether or not a loan should be granted by calculating the probability of default (unable to repay the loan). This assignment requires you to improve the credit scoring algorithm by predicting the probability that somebody will experience financial distress in the next two years.
  
Historical data are provided on 250,000 borrowers. See below dataset.

<b>Content</b>
Training, Test, Sample Entry and Submission Files are provided. Please check the format of the submission file.

<b>Dataset</b>
cs-training.csv
The dataset can be downloaded here: https://github.com/jxchen/Kaggle/blob/master/Give%20Me%20Some%20Credit/cs-training.csv

<br>

In [3]:
file = 'C:/Users/Lenovo/Documents/cs-training.csv'

In [4]:
df = (spark.read.format('csv')
     .option("header", "true")
     .option("mode", "FAILFAST")
     .option("NA", '')
     .load(file))

In [5]:
df.columns

['no',
 'SeriousDlqin2yrs',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents']

In [6]:
#drop the index column to keep relevant features
df = df.drop('no')

In [7]:
from pyspark.sql.functions import regexp_replace
df.withColumn('MonthlyIncome', regexp_replace('MonthlyIncome', 'NA', '0'))

DataFrame[SeriousDlqin2yrs: string, RevolvingUtilizationOfUnsecuredLines: string, age: string, NumberOfTime30-59DaysPastDueNotWorse: string, DebtRatio: string, MonthlyIncome: string, NumberOfOpenCreditLinesAndLoans: string, NumberOfTimes90DaysLate: string, NumberRealEstateLoansOrLines: string, NumberOfTime60-89DaysPastDueNotWorse: string, NumberOfDependents: string]

In [8]:
df.withColumn('NumberOfDependents', regexp_replace('NumberOfDependents', 'NA', '0'))

DataFrame[SeriousDlqin2yrs: string, RevolvingUtilizationOfUnsecuredLines: string, age: string, NumberOfTime30-59DaysPastDueNotWorse: string, DebtRatio: string, MonthlyIncome: string, NumberOfOpenCreditLinesAndLoans: string, NumberOfTimes90DaysLate: string, NumberRealEstateLoansOrLines: string, NumberOfTime60-89DaysPastDueNotWorse: string, NumberOfDependents: string]

In [9]:
df.withColumn('NumberOfTime60-89DaysPastDueNotWorse', regexp_replace('NumberOfTime60-89DaysPastDueNotWorse', 'NA', '0'))

DataFrame[SeriousDlqin2yrs: string, RevolvingUtilizationOfUnsecuredLines: string, age: string, NumberOfTime30-59DaysPastDueNotWorse: string, DebtRatio: string, MonthlyIncome: string, NumberOfOpenCreditLinesAndLoans: string, NumberOfTimes90DaysLate: string, NumberRealEstateLoansOrLines: string, NumberOfTime60-89DaysPastDueNotWorse: string, NumberOfDependents: string]

In [10]:
#dataframe size
df.describe()

DataFrame[summary: string, SeriousDlqin2yrs: string, RevolvingUtilizationOfUnsecuredLines: string, age: string, NumberOfTime30-59DaysPastDueNotWorse: string, DebtRatio: string, MonthlyIncome: string, NumberOfOpenCreditLinesAndLoans: string, NumberOfTimes90DaysLate: string, NumberRealEstateLoansOrLines: string, NumberOfTime60-89DaysPastDueNotWorse: string, NumberOfDependents: string]

In [11]:
#clean data
from pyspark.sql.types import *
from pyspark.sql.functions import col, expr, count

df = df.withColumn("SeriousDlqin2yrs", col("SeriousDlqin2yrs").cast(IntegerType())) \
.withColumn("RevolvingUtilizationOfUnsecuredLines", col("RevolvingUtilizationOfUnsecuredLines").cast(FloatType())) \
.withColumn("age", col("age").cast(IntegerType())) \
.withColumn("NumberOfTime30-59DaysPastDueNotWorse", col("NumberOfTime30-59DaysPastDueNotWorse").cast(IntegerType())) \
.withColumn("DebtRatio", col("DebtRatio").cast(FloatType())) \
.withColumn("MonthlyIncome", col("MonthlyIncome").cast(IntegerType())) \
.withColumn("NumberOfOpenCreditLinesAndLoans", col("NumberOfOpenCreditLinesAndLoans").cast(IntegerType())) \
.withColumn("NumberOfTimes90DaysLate", col("NumberOfTimes90DaysLate").cast(IntegerType())) \
.withColumn("NumberRealEstateLoansOrLines", col("NumberRealEstateLoansOrLines").cast(IntegerType())) \
.withColumn("NumberOfTime60-89DaysPastDueNotWorse", col("NumberOfTime60-89DaysPastDueNotWorse").cast(IntegerType())) \
.withColumn("NumberOfDependents", col("NumberOfDependents").cast(IntegerType()))

In [12]:
#only include distinct rows
df = df.distinct()

In [14]:
df.printSchema()

root
 |-- SeriousDlqin2yrs: integer (nullable = true)
 |-- RevolvingUtilizationOfUnsecuredLines: float (nullable = true)
 |-- age: integer (nullable = true)
 |-- NumberOfTime30-59DaysPastDueNotWorse: integer (nullable = true)
 |-- DebtRatio: float (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- NumberOfOpenCreditLinesAndLoans: integer (nullable = true)
 |-- NumberOfTimes90DaysLate: integer (nullable = true)
 |-- NumberRealEstateLoansOrLines: integer (nullable = true)
 |-- NumberOfTime60-89DaysPastDueNotWorse: integer (nullable = true)
 |-- NumberOfDependents: integer (nullable = true)



### `1.	The overall statistics of overdue credit cards.`

In [59]:
df.select(col('RevolvingUtilizationOfUnsecuredLines').alias('Total_Balance'),
          col('DebtRatio').alias('Debt Ratio'), 
          col('NumberOfOpenCreditLinesAndLoans').alias('No. of open credit lines'),
          col('NumberOfTimes90DaysLate').alias('Overdue more than 90 days')) \
.where(expr("SeriousDlqin2yrs == 1")) \
.summary().toPandas()
#.where(expr("RevolvingUtilizationOfUnsecuredLines > 0")) \
#.where(expr("DebtRatio > 0")) \
#.where(expr("NumberOfOpenCreditLinesAndLoans > 0")) \
#.where(expr('NumberOfTimes90DaysLate > 0')) \

,summary,Total_Balance,Debt Ratio,No. of open credit lines,Overdue more than 90 days
0,count,10009,10009,10009,10009
1,mean,4.373000972503297,295.62012304098215,7.895593965431112,1.9871115995603956
2,stddev,131.94762803990088,1239.3523197167,5.649182149853,11.334643991590621
3,min,0.0,0.0,0,0
4,25%,0.39705762,0.19519162,4,0
5,50%,0.8375812,0.4288889,7,0
6,75%,0.9999999,0.8941059,11,1
7,max,8328.0,38793.0,57,98


### ` 2.	Combined statistics of age and overdue credit card.`

In [60]:
df.select(col('age').alias('Age'),
          col('RevolvingUtilizationOfUnsecuredLines').alias('Total_Balance'),
          col('DebtRatio').alias('Debt Ratio'), 
          col('NumberOfOpenCreditLinesAndLoans').alias('No. of open credit lines'),
          col('NumberOfTimes90DaysLate').alias('Overdue more than 90 days')) \
.where(expr("SeriousDlqin2yrs == 1")) \
.where(expr("age > 0")) \
.summary().toPandas()
#.where(expr("RevolvingUtilizationOfUnsecuredLines > 0")) \
#.where(expr("DebtRatio > 0")) \
#.where(expr("NumberOfOpenCreditLinesAndLoans > 0")) \
#.where(expr('NumberOfTimes90DaysLate > 0')) \

,summary,Age,Total_Balance,Debt Ratio,No. of open credit lines,Overdue more than 90 days
0,count,10009,10009,10009,10009,10009
1,mean,45.947047657108605,4.373000972503297,295.62012304098215,7.895593965431112,1.9871115995603956
2,stddev,12.906584737340896,131.94762803990088,1239.3523197167,5.649182149853,11.334643991590621
3,min,21,0.0,0.0,0,0
4,25%,36,0.39705762,0.19519162,4,0
5,50%,45,0.8375812,0.4288889,7,0
6,75%,54,0.9999999,0.8941059,11,1
7,max,101,8328.0,38793.0,57,98


### `3.	Combined statistics of the number of real estate mortgage and the overdue credit card.`

In [61]:
df.select(col('NumberRealEstateLoansOrLines').alias('No_of_Real_Estate_Mortage'),
          col('RevolvingUtilizationOfUnsecuredLines').alias('Total_Balance'),
          col('DebtRatio').alias('Debt Ratio'), 
          col('NumberOfOpenCreditLinesAndLoans').alias('No. of open credit lines'),
          col('NumberOfTimes90DaysLate').alias('Overdue more than 90 days')) \
.where(expr("SeriousDlqin2yrs == 1")) \
.summary().toPandas()
#.where(expr("RevolvingUtilizationOfUnsecuredLines > 0")) \
#.where(expr("DebtRatio > 0")) \
#.where(expr("NumberOfOpenCreditLinesAndLoans > 0")) \
#.where(expr('NumberOfTimes90DaysLate > 0')) \

,summary,No_of_Real_Estate_Mortage,Total_Balance,Debt Ratio,No. of open credit lines,Overdue more than 90 days
0,count,10009,10009,10009,10009,10009
1,mean,0.9902088120691378,4.373000972503297,295.62012304098215,7.895593965431112,1.9871115995603956
2,stddev,1.4263507511079203,131.94762803990088,1239.3523197167,5.649182149853,11.334643991590621
3,min,0,0.0,0.0,0,0
4,25%,0,0.39705762,0.19519162,4,0
5,50%,1,0.8375812,0.4288889,7,0
6,75%,2,0.9999999,0.8941059,11,1
7,max,29,8328.0,38793.0,57,98


### `4.	Combined statistics of the number of family members and the overdue credit card.`

In [62]:
df.select(col('NumberOfDependents').alias('No_of_dependents'),
          col('RevolvingUtilizationOfUnsecuredLines').alias('Total_Balance'),
          col('DebtRatio').alias('Debt Ratio'), 
          col('NumberOfOpenCreditLinesAndLoans').alias('No. of open credit lines'),
          col('NumberOfTimes90DaysLate').alias('Overdue more than 90 days')) \
.where(expr("SeriousDlqin2yrs == 1")) \
.summary().toPandas()
#.where(expr("RevolvingUtilizationOfUnsecuredLines > 0")) \
#.where(expr("DebtRatio > 0")) \
#.where(expr("NumberOfOpenCreditLinesAndLoans > 0")) \
#.where(expr('NumberOfTimes90DaysLate > 0')) \

,summary,No_of_dependents,Total_Balance,Debt Ratio,No. of open credit lines,Overdue more than 90 days
0,count,9831,10009,10009,10009,10009
1,mean,0.9497507883226528,4.373000972503297,295.62012304098215,7.895593965431112,1.9871115995603956
2,stddev,1.2197583641361514,131.94762803990088,1239.3523197167,5.649182149853,11.334643991590621
3,min,0,0.0,0.0,0,0
4,25%,0,0.39705762,0.19519162,4,0
5,50%,0,0.8375812,0.4288889,7,0
6,75%,2,0.9999999,0.8941059,11,1
7,max,8,8328.0,38793.0,57,98


### `5.	Combined statistics of monthly income and overdue credit card.`

In [63]:
df.select(col('MonthlyIncome').alias('MonthlyIncome'),
          col('RevolvingUtilizationOfUnsecuredLines').alias('Total_Balance'),
          col('DebtRatio').alias('Debt Ratio'), 
          col('NumberOfOpenCreditLinesAndLoans').alias('No. of open credit lines'),
          col('NumberOfTimes90DaysLate').alias('Overdue more than 90 days')) \
.where(expr("SeriousDlqin2yrs == 1")) \
.summary().toPandas()
#.where(expr("MonthlyIncome > 0")) \
#.where(expr("RevolvingUtilizationOfUnsecuredLines > 0")) \
#.where(expr("DebtRatio > 0")) \
#.where(expr("NumberOfOpenCreditLinesAndLoans > 0")) \
#.where(expr('NumberOfTimes90DaysLate > 0')) \

,summary,MonthlyIncome,Total_Balance,Debt Ratio,No. of open credit lines,Overdue more than 90 days
0,count,8355,10009,10009,10009,10009
1,mean,5632.054697785757,4.373000972503297,295.62012304098215,7.895593965431112,1.9871115995603956
2,stddev,6171.942881557316,131.94762803990088,1239.3523197167,5.649182149853,11.334643991590621
3,min,0,0.0,0.0,0,0
4,25%,2963,0.39705762,0.19519162,4,0
5,50%,4500,0.8375812,0.4288889,7,0
6,75%,6800,0.9999999,0.8941059,11,1
7,max,250000,8328.0,38793.0,57,98


### `6.	Statistics of overdue users of different ages.`

In [64]:
overdue_age = df.select('age') \
.filter(expr('SeriousDlqin2yrs == 1'))
#.filter(expr('RevolvingUtilizationOfUnsecuredLines > 0 and DebtRatio > 0 and NumberOfOpenCreditLinesAndLoans > 0 and SeriousDlqin2yrs == 1'))

In [65]:
grouped = overdue_age.groupBy('age') \
.agg(count(col('age')).alias('Overdue users')) \
.filter(expr('age > 0')) \
.orderBy('Overdue users', ascending=False)

In [66]:
grouped.show(100)

+---+-------------+
|age|Overdue users|
+---+-------------+
| 46|          324|
| 49|          313|
| 47|          306|
| 41|          294|
| 42|          289|
| 48|          286|
| 45|          284|
| 39|          280|
| 50|          279|
| 52|          277|
| 43|          276|
| 51|          273|
| 53|          264|
| 40|          262|
| 33|          246|
| 54|          243|
| 44|          242|
| 35|          241|
| 36|          237|
| 38|          235|
| 32|          233|
| 37|          227|
| 55|          218|
| 31|          216|
| 34|          210|
| 30|          208|
| 56|          207|
| 28|          205|
| 57|          182|
| 29|          179|
| 61|          175|
| 59|          167|
| 27|          165|
| 58|          164|
| 62|          163|
| 60|          150|
| 26|          147|
| 63|          136|
| 25|          119|
| 24|           96|
| 64|           94|
| 65|           92|
| 66|           80|
| 67|           79|
| 23|           68|
| 70|           52|
| 72|           46|


### `7.	Among the overdue users, how many people have monthly income of less than 10,000 and have real estate loans, and what is the total proportion?`

In [67]:
monthly_estate = df.select(df.columns) \
.filter(expr('SeriousDlqin2yrs == 1')) \
.where(expr('MonthlyIncome < 10000')) \
.where(expr('NumberRealEstateLoansOrLines > 0')) \
.count()
monthly_estate
#.filter(expr('RevolvingUtilizationOfUnsecuredLines > 0 and DebtRatio > 0 and NumberOfOpenCreditLinesAndLoans > 0 and SeriousDlqin2yrs == 1')) \

3924

In [24]:
monthly_estate/df.count()

0.025416524422488637

### `8.	Among the overdue users, how many people with monthly income of less than 10,000 and have more than 3 family members, and what is the total proportion?`

In [68]:
monthly_fam = df.select(df.columns) \
.filter(expr('SeriousDlqin2yrs == 1')) \
.where(expr('SeriousDlqin2yrs != 0')) \
.where(expr('MonthlyIncome < 10000')) \
.where(expr('NumberOfDependents > 3')) \
.count()
monthly_fam
#.filter(expr('RevolvingUtilizationOfUnsecuredLines > 0 and DebtRatio > 0 and NumberOfOpenCreditLinesAndLoans > 0 and SeriousDlqin2yrs == 1')) \

310

In [69]:
monthly_fam/df.count()

0.002075091538312214

### `9.	What is the total amount of credit card outstanding among all overdue users?`

In [70]:
df_credit = df.select(col('SeriousDlqin2yrs'), col('RevolvingUtilizationOfUnsecuredLines')*col('DebtRatio')*col('NumberOfOpenCreditLinesAndLoans')*col('NumberOfTimes90DaysLate')) \
.filter(expr('SeriousDlqin2yrs != 0'))

In [71]:
rdd = df_credit.rdd
rdd = rdd.map(lambda x: (x[0], x[1])) 
rdd.map(lambda x: x[1]) \
.filter(lambda x: float(x) > 0.0) \
.count()
#.reduce(lambda x,y: x+y)

3030